In [15]:
import cv2
import numpy as np
import sys
from random import randint
import math

video_path = 'data/kart.avi'

cv2.namedWindow("LineTrackingWindow")
cap = cv2.VideoCapture(video_path)
Multitracker = cv2.MultiTracker_create()     # MultiTracker 객체 생성 

# 첫번째 프레임 읽기
ok, frame = cap.read()
if not ok:
    print('Failed to read video')
    exit()    # 커널 종료 
    
bboxes = []
n = 0
while n<1:       # ROI 3번 설정 
    bbox = cv2.selectROI('LineTrackingWindow', frame)   # BoundingBox 설정 
    bboxes.append(bbox)
    n += 1

for bbox in bboxes:
    Multitracker.add(cv2.TrackerMIL_create(), frame, bbox)  # Multitracker.add(알고리즘, 이미지, bbox)
    
'''
OPENCV_OBJECT_TRACKERS = {
"csrt": cv2.TrackerCSRT_create,      # More accurate than KCF but slower. 낮은 프레임(fps25)에서 사용 (Best)
"kcf": cv2.TrackerKCF_create,        # Fast and accurate. but 가림현상을 잘 처리하지 못함.
"boosting": cv2.TrackerBoosting_create,  # 오래된 추적기로 성능이 떨어짐.
"mil": cv2.TrackerMIL_create,            # boosting에 비해 정확도는 좋으나, 실패보고가 정상적으로 안됨.
"tld": cv2.TrackerTLD_create,       # 가림현상에 잘 작동하나, Android버전에서 에러빈번하게 발생.
"medianflow": cv2.TrackerMedianFlow_create,   # 실패보고가 잘 작동하나, 빠른 변화에 취약함 
"mosse": cv2.TrackerMOSSE_create,     # Extremely fast but not as accurate as either KCF or CSRT
"goturn": cv2.TrackerGOTURN_create    # 딥러닝 기반. 추가 모델 필요.
}
'''

colors = []
colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )      # 랜덤컬러 3번설정 
#colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )
#colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )
print(colors)

while cap.isOpened():
    ok, frame = cap.read()        # 동영상을 1프레임씩 읽어온다. -> frame 변수에 넣는다.
                                # 동영상 끝나면 ok = False
    if not ok:
        print ('no image to read')
        break
    
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    frame2 = cv2.resize(frame, (640, 360))           # 결과창을 (640, 360)으로  ReSize
    dst = cv2.Canny(frame2, 500, 700, None, 3)       # Canny Edge Detection Algorithm 
                                                    # cv2.Canny (inputImage, minVal, maxVal)
    
    cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)     # GRAY -> BGR 인 image 생성
    cdstP = np.copy(cdst)
    
    # Line 1 : Standard Hough Line Transform
    lines = cv2.HoughLines(dst, 1, np.pi / 180, 170)  

    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))    # FirstPoint of line Segment
            pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))    # SecondPoint of line Segment
            cv2.line(cdst, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)   # cv2.line(img, pt1, pt2, color, thickness, lineType)
    
    # Line 2 : Probabilistic Line Transform
    linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            pt1 = (l[0], l[1])
            pt2 = (l[2], l[3])
            cv2.line(cdstP, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)  
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    
    ## Start timer
    timer = cv2.getTickCount()

    ok, boxes = Multitracker.update(frame)  # frame에서 설정한 boundingBox와 비슷한 물체의 위치를 
                                            # 찾아(Update) 반환한다.
    
    ## Calculate Frames per second (FPS) 
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    
    i = 0
    for newbox in boxes:
        # newbox의 왼쪽, 위쪽, 너비, 높이
        left, top, w, h = [int(v) for v in newbox]
        
        # boxes의 색깔,두께 설정
        cv2.rectangle(frame, pt1=(left, top), pt2=(left + w, top + h), color=colors[i], thickness=3)   # 이미지에 바운딩박스 그린다.
        i += 1

    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (270,100), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)
    
    cv2.imshow('LineTrackingWindow', frame)
    
    cv2.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
    cv2.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdstP)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        


cap.release()
cv2.destroyAllWindows()

[(202, 65, 168)]


## reference
```
Canny Edge Detection : <https://m.blog.naver.com/samsjang/220507996391>
Hough Transform (허프 변환) : <https://opencv-python.readthedocs.io/en/latest/doc/25.imageHoughLineTransform/imageHoughLineTransform.html>

허프변환의 단점은 명확하다. 곡선에서는 사용이 불가능하며 차선이 아닌 직선도 차선으로 인식한다는 것이다.
HSV를 이용한 필터링과 ROI로 영역을 나누면 꽤 쓸만해진다.
이렇게 ROI로 영역을 선택하고 HSV 필터링을 이용해 차선만 남기고 제거하면 꽤 정확한 차선인식이 가능하다. 
그리고 대부분의 노이즈도 제거가 가능하고 곡선도 어느정도는 검출이 가능하다.
```

In [8]:
cv2.HoughLines?